In [ ]:
import sys
sys.path.append("..")
from Utils import VariantWordDataset

train_set = VariantWordDataset("train", "../Data/source_vocal.pkl", "../Data/target_vocal.pkl")
print(train_set.__len__())
train_set.__getitem__(1)
len(train_set.target_dic.word2idx)


In [ ]:
import torch
from torch import Tensor
import numpy as np
a = torch.Tensor([[   1,    10],
        [ 122, 2138],
        [3420, 3560],
        [3155, 1954],
        [1897,  835],
        [ 453, 1366],
        [1881,    5],
        [1938, 2084],
        [3557,  837],
        [1711, 1504],
        [  41, 3202],
        [1377,  716],
        [2351, 3359]])
b = torch.ones(a.shape[0])
# print(b)
b = b.unsqueeze(1)
a = torch.cat([a,b],1)
a
length = [2,1,1,1,1, 1,1,1,1,1, 1,1,1]
b = []
for i,j in zip(a,length):
        print(i[:j])
        b.append(i[:j].flip(0))
        print(b)
        break

In [ ]:
from torch.utils.data import DataLoader
# from torch.utils.data.dataloader import default_collate
import numpy as np
import torch


def generate_square_subsequent_mask(tgt_seq_len, device):

    mask = (torch.triu(torch.ones((tgt_seq_len, tgt_seq_len), device=device)) == 1).transpose(0, 1)

    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

    return mask

def create_mask( src, tgt, device='cpu'):
    src_seq_len = src.shape[0]

    tgt_seq_len = tgt.shape[0]

    # Decoder的注意力Mask输入，用于掩盖当前position之后的position，所以这里是一个对称矩阵
    tgt_mask = generate_square_subsequent_mask(tgt_seq_len, device)  # [tgt_len,tgt_len]

    # Encoder的注意力Mask输入，这部分其实对于Encoder来说是没有用的，所以这里全是0
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=device).type(torch.bool)

    # 用于mask掉Encoder的Token序列中的padding部分,[batch_size, src_len]
    src_padding_mask = (src == 0).transpose(0, 1)

    # 用于mask掉Decoder的Token序列中的padding部分,batch_size, tgt_len
    tgt_padding_mask = (tgt == 0).transpose(0, 1)

    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask



train_loader = DataLoader(train_set, batch_size=2, collate_fn=train_set.generate_batch,
                                               shuffle=False)
for src, tgt in train_loader:
    print(src)
    print(src.shape)
    src = src.transpose(0,1)
    print(src)
    print(src.shape)
    # tgt_input = tgt[:-1, :]

    # tgt_out = tgt[1:, :]

    # src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

    
    # print("src shape:", src.shape)  # [de_tensor_len,batch_size]

    # print("src_padding_mask shape (batch_size, src_len): ", src_padding_mask.shape)

    # print("tgt input shape:", tgt_input.shape)

    # print("tgt_padding_mask shape: (batch_size, tgt_len) ", tgt_padding_mask.shape)
    # print("tgt output shape:", tgt_out.shape)

    # print("tgt_mask shape (tgt_len,tgt_len): ", tgt_mask.shape)
    # print("src_mask shape (tgt_len,tgt_len): ", src_mask.shape)
    # print("src_padding_mask shape (batch_size, src_len): ", src_padding_mask.shape)

    break



In [ ]:
from Model.TranslationModel import TranslationModel
from Model.config import Config
from torchtext.data.metrics import bleu_score
from Utils.Dictionary import Dictionary

config = Config()
dic = Dictionary.load_from_file(config.target_dic_path)

translation_model = TranslationModel(src_vocab_size=len(train_set.source_dic.word2idx),
                                         tgt_vocab_size=len(train_set.target_dic.word2idx),
                                         d_model=config.d_model,
                                         nhead=config.num_head,
                                         num_encoder_layers=config.num_encoder_layers,
                                         num_decoder_layers=config.num_decoder_layers,
                                         dim_feedforward=config.dim_feedforward,
                                         dropout=config.dropout)

def translate(tensor):
    result = [dic.idx2word[i] for i in tensor.numpy()]
    return result


for src, tgt in train_loader:
    src = src.to(config.device)  # [src_len, batch_size]
    tgt = tgt.to(config.device)
    tgt_input = tgt[:-1, :]  # 解码部分的输入, [tgt_len,batch_size]
    src_mask, tgt_mask, src_padding_mask, tgt_padding_mask \
    = create_mask(src, tgt_input, config.device)
    logits = translation_model(
    src=src,  # Encoder的token序列输入，[src_len,batch_size]
    tgt=tgt_input,  # Decoder的token序列输入,[tgt_len,batch_size]
    src_mask=src_mask,  # Encoder的注意力Mask输入，这部分其实对于Encoder来说是没有用的
    tgt_mask=tgt_mask,
    # Decoder的注意力Mask输入，用于掩盖当前position之后的position [tgt_len,tgt_len]
    src_key_padding_mask=src_padding_mask,  # 用于mask掉Encoder的Token序列中的padding部分
    tgt_key_padding_mask=tgt_padding_mask,  # 用于mask掉Decoder的Token序列中的padding部分
    memory_key_padding_mask=src_padding_mask)  # 用于mask掉Encoder的Token序列中的padding部分

    tgt_out = tgt[1:, :]  # 解码部分的真实值  shape: [tgt_len,batch_size]
    # logits 输出shape为[tgt_len,batch_size,tgt_vocab_size]
    # bleu_score(tgt_out.t().squeeze(), tgt_input.t().squeeze())


    print(logits.reshape(-1, logits.shape[-1]))
    # for i, j in zip(tgt_out.t(), tgt_input.t()):
    #     i = [translate(i)]
    #     j = [[translate(j)]]
    #     print(i)
    #     print(j)
    #     print(bleu_score(i, j))
    break

In [ ]:
# from config import Config
from Model.TranslationModel import TranslationModel
import torch
import time
import os
import logging
from tqdm import tqdm
def accuracy(logits, y_true, PAD_IDX):
    """
    :param logits:  [tgt_len,batch_size,tgt_vocab_size]
    :param y_true:  [tgt_len,batch_size]
    :param PAD_IDX:
    :return:
    """
    y_pred = logits.transpose(0, 1).argmax(axis=2).reshape(-1)
    # 将 [tgt_len,batch_size,tgt_vocab_size] 转成 [batch_size, tgt_len,tgt_vocab_size]
    y_true = y_true.transpose(0, 1).reshape(-1)
    # 将 [tgt_len,batch_size] 转成 [batch_size， tgt_len]
    acc = y_pred.eq(y_true)  # 计算预测值与正确值比较的情况
    mask = torch.logical_not(y_true.eq(PAD_IDX))  # 找到真实标签中，mask位置的信息。 mask位置为FALSE，非mask位置为TRUE
    acc = acc.logical_and(mask)  # 去掉acc中mask的部分
    correct = acc.sum().item()
    total = mask.sum().item()
    return float(correct) / total, correct, total

def train_model(config):
    logging.info("############载入数据集############")

    train_set = VariantWordDataset("train", "../Data/source_vocal.pkl", "../Data/target_vocal.pkl")
    valid_set = VariantWordDataset("test", "../Data/source_vocal.pkl", "../Data/target_vocal.pkl")


    logging.info("############划分数据集############")

    train_set_loader = DataLoader(train_set, batch_size=config.batch_size, 
                                    collate_fn=train_set.generate_batch, shuffle=False)

    valid_set_loader = DataLoader(valid_set, batch_size=config.batch_size, 
                                    collate_fn=valid_set.generate_batch, shuffle=False)


    logging.info("############初始化模型############")
    translation_model = TranslationModel(src_vocab_size=len(train_set.source_dic.word2idx),
                                         tgt_vocab_size=len(train_set.target_dic.word2idx),
                                         d_model=config.d_model,
                                         nhead=config.num_head,
                                         num_encoder_layers=config.num_encoder_layers,
                                         num_decoder_layers=config.num_decoder_layers,
                                         dim_feedforward=config.dim_feedforward,
                                         dropout=config.dropout)

    

    model_save_path = os.path.join(config.model_save_dir, 'model.pkl')

    if os.path.exists(model_save_path):
        loaded_paras = torch.load(model_save_path)
        translation_model.load_state_dict(loaded_paras)
        logging.info("#### 成功载入已有模型，进行追加训练...")


    translation_model = translation_model.to(config.device)


    loss_fn = torch.nn.CrossEntropyLoss(ignore_index=train_set.PAD_IDX)
    optimizer = torch.optim.Adam(translation_model.parameters(),
                                 lr=0.,
                                 betas=(config.beta1, config.beta2), eps=config.epsilon)

    # lr_scheduler = CustomSchedule(config.d_model, optimizer=optimizer)

    translation_model.train()
    for epoch in  range(config.epochs):
        losses = 0
        start_time = time.time()
        for idx, (src, tgt) in tqdm(enumerate(train_set_loader)):
            src = src.to(config.device)  # [src_len, batch_size]
            tgt = tgt.to(config.device)
            tgt_input = tgt[:-1, :]  # 解码部分的输入, [tgt_len,batch_size]
            src_mask, tgt_mask, src_padding_mask, tgt_padding_mask \
                = create_mask(src, tgt_input, config.device)
            logits = translation_model(
                src=src,  # Encoder的token序列输入，[src_len,batch_size]
                tgt=tgt_input,  # Decoder的token序列输入,[tgt_len,batch_size]
                src_mask=src_mask,  # Encoder的注意力Mask输入，这部分其实对于Encoder来说是没有用的
                tgt_mask=tgt_mask,
                # Decoder的注意力Mask输入，用于掩盖当前position之后的position [tgt_len,tgt_len]
                src_key_padding_mask=src_padding_mask,  # 用于mask掉Encoder的Token序列中的padding部分
                tgt_key_padding_mask=tgt_padding_mask,  # 用于mask掉Decoder的Token序列中的padding部分
                memory_key_padding_mask=src_padding_mask)  # 用于mask掉Encoder的Token序列中的padding部分
            # logits 输出shape为[tgt_len,batch_size,tgt_vocab_size]
            optimizer.zero_grad()
            tgt_out = tgt[1:, :]  # 解码部分的真实值  shape: [tgt_len,batch_size]
            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
            # [tgt_len*batch_size, tgt_vocab_size] with [tgt_len*batch_size, ]
            loss.backward()
            # lr_scheduler.step()
            optimizer.step()
            losses += loss.item()
            acc, _, _ = accuracy(logits, tgt_out, train_set.PAD_IDX)
            msg = f"Epoch: {epoch}, Batch[{idx}/{len(train_set_loader)}], Train loss :{loss.item():.3f}, Train acc: {acc}"
            logging.info(msg)
        end_time = time.time()
        train_loss = losses / len(train_set_loader)
        msg = f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Epoch time = {(end_time - start_time):.3f}s"
        logging.info(msg)



        if epoch % 2 == 0:
            acc = evaluate(config, valid_set_loader, translation_model, dataset)
            logging.info(f"Accuracy on validation{acc:.3f}")
            torch.save(translation_model.state_dict(), model_save_path)


def evaluate(config, valid_iter, model, dataset):
    model.eval()
    correct, totals = 0, 0
    with torch.no_grad():
        for idx, (src, tgt) in enumerate(valid_iter):
            src = src.to(config.device)
            tgt = tgt.to(config.device)
            tgt_input = tgt[:-1, :]  # 解码部分的输入

            src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = \
                create_mask(src, tgt_input, device=config.device)

            logits = model(src=src,  # Encoder的token序列输入，
                           tgt=tgt_input,  # Decoder的token序列输入
                           src_mask=src_mask,  # Encoder的注意力Mask输入，这部分其实对于Encoder来说是没有用的
                           tgt_mask=tgt_mask,  # Decoder的注意力Mask输入，用于掩盖当前position之后的position
                           src_key_padding_mask=src_padding_mask,  # 用于mask掉Encoder的Token序列中的padding部分
                           tgt_key_padding_mask=tgt_padding_mask,  # 用于mask掉Decoder的Token序列中的padding部分
                           memory_key_padding_mask=src_padding_mask)  # 用于mask掉Encoder的Token序列中的padding部分
            tgt_out = tgt[1:, :]  # 解码部分的真实值  shape: [tgt_len,batch_size]
            _, c, t = accuracy(logits, tgt_out, valid_iter.PAD_IDX)
            correct += c
            totals += t
    model.train()
    return float(correct) / totals

In [ ]:
import sys
sys.path.append("..")
from Model.config import Config
config = Config()
train_model(config)

In [ ]:
import sys
sys.path.append("..")
from Model.TranslationModel import TranslationModel
from Model.config import Config
import torch
config = Config()
translation_model = TranslationModel(src_vocab_size=config.source_vocab_size,
                                        tgt_vocab_size=config.target_vocab_size,
                                        d_model=config.d_model,
                                        nhead=config.num_head,
                                        num_encoder_layers=config.num_encoder_layers,
                                        num_decoder_layers=config.num_decoder_layers,
                                        dim_feedforward=config.dim_feedforward,
                                        dropout=config.dropout)

load_param = torch.load("../cache/model.pkl", map_location=torch.device('cpu'))
translation_model.load_state_dict(load_param)



In [ ]:
from Utils.Dictionary import Dictionary
dic = Dictionary.load_from_file("../Data/source_vocal.pkl")
len(dic.word2idx)
dic.word2idx
dic.__len__()

In [ ]:
import numpy as np
from torchtext.data.metrics import bleu_score
# candidate_corpus = [['权', '益', '服', '务', ':', '您', '领', '取', '的', '双', '突', '战', '法', '指']]
# references_corpus = [[['权', '益', '服', '务', ':', '您', '领', '取', '的', '双', '突', '战', '法']]]
# candidate_corpus = ['权', '益', '服', '务']
# references_corpus = ['权', '益', '服', '务']
candidate_corpus = [['权', '益', '服', '务', ':', '您', '领', '取', '的', '双', '突', '战', '法', '指', '标', '今', '天', '已', '有', '信', '号', '提', '醒', '啦', ',', '快', '登', '录', '大', '智', '慧', 'b', '查', '看', '或', '现', '在', '打', '开', '链', '接', 'h', ':', '/', '/', 't', '.', 'v', '.', 'e', '.', 'c', 'd', '/', 'q', '使', '用', '.', '[EOS]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']]
references_corpus = [[['权', '益', '服', '务', ':', '您', '领', '取', '的', '双', '突', '战', '法', '指', '标', '今', '天', '已', '有', '信', '号', '提', '醒', '啦', ',', '快', '登', '录', '大', '智', '慧', 'b', '查', '看', '或', '现', '在', '打', '开', '链', '接', 'h', ':', '/', '/', 't', '.', 'v', '.', 'e', '.', 'c', 'd', '/', 'm', '使', '用', '.', '[EOS]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']]]

print(len(candidate_corpus[0]))
print(len(references_corpus[0]))
print(references_corpus[0][0])
bleu_score(candidate_corpus, references_corpus)

In [ ]:
import torch
import numpy as np
a = torch.Tensor([[10,11,12,13,14,15],[10,11,12,13,14,15]])
a.shape
a.sum(0)

In [ ]:
import torch
import torch.nn.functional as F 
a = torch.Tensor([[1],[2],[3]])
b = torch.Tensor([[1],[2],[3]])
torch.cat((a,b), 1)
a.shape[0]
a.size(0)
torch.zeros(8,1,device="cpu")
a = torch.Tensor([[0.3, 0.7]])
F.softmax(a)

### 计算原始数据集的BLEU分数

In [14]:
import sys
sys.path.append("..")
import pandas as pd
from data_loader import Dictionary
from nltk.translate import bleu_score

source_dic =Dictionary.load_from_file("../Data/source_vocal.pkl")
target_dic =Dictionary.load_from_file("../Data/target_vocal.pkl")


test_data = pd.read_csv("../Data/Dataset/test_data.csv",index_col=0)
raw_data = test_data["raw_data"].tolist()
right_data = test_data["right_data"].tolist()

# def replace(sentence):

def C_trans_to_E(string):
    E_pun = u',.!?[]()<>"\':;'
    C_pun = u'，。！？【】（）《》“‘：；'
    table= {ord(f):ord(t) for f,t in zip(C_pun,E_pun)}
    return string.translate(table)


raw_data2idx = []
right_data2idx = []

# 直接构建
for source, target in zip(raw_data, right_data):
    raw_data2idx.append(list(source))
    right_data2idx.append([list(target)])

bleuscore_1 =  bleu_score.corpus_bleu(right_data2idx, raw_data2idx)

raw_data2idx.clear()
right_data2idx.clear()

# C2E构建
for source, target in zip(raw_data, right_data):
    raw_data2idx.append(list(C_trans_to_E(source)))
    right_data2idx.append([list(C_trans_to_E(target))])
bleuscore_2=  bleu_score.corpus_bleu(right_data2idx, raw_data2idx)


loading dictionary from ../Data/source_vocal.pkl
loading dictionary from ../Data/target_vocal.pkl


In [15]:
print("直接构建BLEU_SCORE:", bleuscore_1)
print("C2E构建BLEU_SCORE:", bleuscore_2)
# print("C2E构建BLEU_SCORE:", bleuscore_3)

直接构建BLEU_SCORE: 0.6048509912936112
C2E构建BLEU_SCORE: 0.7032748700029685


In [7]:
def E_trans_to_C(string):
    E_pun = u',.!?[]()<>"\':;'
    C_pun = u'，。！？【】（）《》“‘：；'
    table= {ord(f):ord(t) for f,t in zip(C_pun,E_pun)}
    return string.translate(table)

test = "昨晚开出(46_虎)，您鈡了嚒?加wei：18683367080 我可以告诉你下次的壹逍叁玛，早加就知道"
print(E_trans_to_C(test))

昨晚开出(46_虎),您鈡了嚒?加wei:18683367080 我可以告诉你下次的壹逍叁玛,早加就知道
